In [1]:
# Import dependencies
import pandas as pd
import numpy as np

### Read in City Data from  CSV (US Census Data)

In [2]:
# Read the data into a Pandas DataFrame
city_info_df = pd.read_excel('Resources/city_data.xlsx')
city_info_df.head()

,Rank,"City, State",Drop 20,Drop 20.1,Drop 21,Drop 22,Recent-2023
0,1,"New York city, New York",8804199.0,8740292.0,8462216.0,8335798.0,8258035.0
1,2,"Los Angeles city, California",3898841.0,3895848.0,3832573.0,3822782.0,3820914.0
2,3,"Chicago city, Illinois",2746352.0,2743329.0,2704101.0,2672660.0,2664452.0
3,4,"Houston city, Texas",2300833.0,2299269.0,2291020.0,2302488.0,2314157.0
4,5,"Phoenix city, Arizona",1608215.0,1612459.0,1625187.0,1643899.0,1650070.0


In [3]:
# Get a brief summary of the city_info DataFrame.
city_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1915 entries, 0 to 1914
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         1915 non-null   object 
 1   City, State  1909 non-null   object 
 2   Drop 20      1909 non-null   float64
 3   Drop 20.1    1909 non-null   float64
 4   Drop 21      1909 non-null   float64
 5   Drop 22      1909 non-null   float64
 6   Recent-2023  1909 non-null   float64
dtypes: float64(5), object(2)
memory usage: 104.9+ KB


### Clean City Data

1) Create a city and state column
2) Change Dtypes
3) Update column names
4) Delete columns not needed
5) Create city ids
6) Clean to only NFL stadium cities

In [4]:
# Create city and state column
city_info_df[['City', 'State']] = city_info_df['City, State'].str.split(',', expand=True)
city_info_df.head()

,Rank,"City, State",Drop 20,Drop 20.1,Drop 21,Drop 22,Recent-2023,City,State
0,1,"New York city, New York",8804199.0,8740292.0,8462216.0,8335798.0,8258035.0,New York city,New York
1,2,"Los Angeles city, California",3898841.0,3895848.0,3832573.0,3822782.0,3820914.0,Los Angeles city,California
2,3,"Chicago city, Illinois",2746352.0,2743329.0,2704101.0,2672660.0,2664452.0,Chicago city,Illinois
3,4,"Houston city, Texas",2300833.0,2299269.0,2291020.0,2302488.0,2314157.0,Houston city,Texas
4,5,"Phoenix city, Arizona",1608215.0,1612459.0,1625187.0,1643899.0,1650070.0,Phoenix city,Arizona


In [5]:
# Drop City,State and Columns for 2020, 2021, and 2022 (we are only using 2023 metrics)
city_info_df = city_info_df.drop(['City, State', 'Drop 20', 'Drop 20.1','Drop 21', 'Drop 22', 'Rank'], axis=1)

# Reorder
city_info_df = city_info_df[['City', 'State', 'Recent-2023']]

#Rename 2023 column
city_info_df = city_info_df.rename(columns={'Recent-2023': 'Population Est 2023'})

city_info_df

,City,State,Population Est 2023
0,New York city,New York,8258035.0
1,Los Angeles city,California,3820914.0
2,Chicago city,Illinois,2664452.0
3,Houston city,Texas,2314157.0
4,Phoenix city,Arizona,1650070.0
...,...,...,...
1910,NaN,NaN,NaN
1911,NaN,NaN,NaN
1912,NaN,NaN,NaN
1913,NaN,NaN,NaN


In [6]:
# Drop NaN
city_info_df = city_info_df.dropna()
city_info_df

,City,State,Population Est 2023
0,New York city,New York,8258035.0
1,Los Angeles city,California,3820914.0
2,Chicago city,Illinois,2664452.0
3,Houston city,Texas,2314157.0
4,Phoenix city,Arizona,1650070.0
...,...,...,...
1904,Plainview city,Texas,19420.0
1905,Harvey city,Illinois,19397.0
1906,Ypsilanti city,Michigan,19393.0
1907,Albany city,California,19097.0


In [7]:
# Drop "city" from city column
city_info_df.loc[:, 'City'] = city_info_df['City'].str.replace(r'\s+city$', '', regex=True)
city_info_df

,City,State,Population Est 2023
0,New York,New York,8258035.0
1,Los Angeles,California,3820914.0
2,Chicago,Illinois,2664452.0
3,Houston,Texas,2314157.0
4,Phoenix,Arizona,1650070.0
...,...,...,...
1904,Plainview,Texas,19420.0
1905,Harvey,Illinois,19397.0
1906,Ypsilanti,Michigan,19393.0
1907,Albany,California,19097.0


In [8]:
# Convert Data Types
city_info_df = city_info_df.copy()

# Convert 'City' and 'State' to string
city_info_df['City'] = city_info_df['City'].astype(str)
city_info_df['State'] = city_info_df['State'].astype(str)

# Convert 'Population Est 2023' to int, handling NaN values
city_info_df['Population Est 2023'] = pd.to_numeric(city_info_df['Population Est 2023'], errors='coerce').fillna(0).astype(int)

city_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1909 entries, 0 to 1908
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   City                 1909 non-null   object
 1   State                1909 non-null   object
 2   Population Est 2023  1909 non-null   int32 
dtypes: int32(1), object(2)
memory usage: 52.2+ KB


In [9]:
# Filter for NFL cities
nfl_cities = ['Arlington','Atlanta','Baltimore','Charlotte','Chicago','Cincinnati','Cleveland','Denver',
             '	Detroit','East Rutherford','Foxborough','New York','Boston','Clendale','Green Bay','Houston',
             'Indianapolis','Inglewood','Los Angeles', 'Jacksonville', 'Kansas City','Landover', 'Washington',
             'Miami Gardens','Miami','Minneapolis','Nashville','New Orleans','Orchard Park','Buffalo', 'Paradise',
             'Las Vegas','	Philadelphia','Pittsburgh','Santa Clara','San Francisco','Seattle','Tampa']
nfl_city_info_df = city_info_df.query("City in @nfl_cities")

nfl_city_info_df

,City,State,Population Est 2023
0,New York,New York,8258035
1,Los Angeles,California,3820914
2,Chicago,Illinois,2664452
3,Houston,Texas,2314157
9,Jacksonville,Florida,985843
14,Charlotte,North Carolina,911311
16,San Francisco,California,808988
17,Seattle,Washington,755078
18,Denver,Colorado,716577
21,Washington,District of Columbia,678972


In [10]:
# Drop duplicate name cities
nfl_city_info_df = nfl_city_info_df.drop([514, 820, 1190,1360,1801]) 

nfl_city_info_df

,City,State,Population Est 2023
0,New York,New York,8258035
1,Los Angeles,California,3820914
2,Chicago,Illinois,2664452
3,Houston,Texas,2314157
9,Jacksonville,Florida,985843
14,Charlotte,North Carolina,911311
16,San Francisco,California,808988
17,Seattle,Washington,755078
18,Denver,Colorado,716577
21,Washington,District of Columbia,678972


In [11]:
def generate_unique_id(city, state):
    # Get the first letter of each word in the city name
    city_initials = ''.join(word[0].upper() for word in city.split())
    
    # Get the first letter of each word in the state name
    state_initials = ''.join(word[0].upper() for word in state.split())
    
    # Combine city initials and state initials
    return f"{city_initials}{state_initials}"

# Create a dictionary to keep track of ID occurrences
id_count = {}

def get_unique_id(row):
    base_id = generate_unique_id(row['City'], row['State'])
    if base_id in id_count:
        id_count[base_id] += 1
        return f"{base_id}{id_count[base_id]}"
    else:
        id_count[base_id] = 0
        return base_id

# Apply the function to create Unique_ID
nfl_city_info_df['Unique_ID'] = nfl_city_info_df.apply(get_unique_id, axis=1)

# Display the first few rows to verify
print(nfl_city_info_df[['City', 'State', 'Unique_ID']].head(10))

# Display all unique IDs
print("\nAll Unique IDs:")
for _, row in nfl_city_info_df.iterrows():
    print(f"{row['City']}, {row['State']}: {row['Unique_ID']}")

# Check for any duplicate IDs
duplicate_ids = nfl_city_info_df[nfl_city_info_df.duplicated(subset='Unique_ID', keep=False)]
if not duplicate_ids.empty:
    print("\nWarning: Some cities have duplicate IDs:")
    print(duplicate_ids[['City', 'State', 'Unique_ID']])
else:
    print("\nAll IDs are unique.")

             City                  State Unique_ID
0        New York               New York      NYNY
1     Los Angeles             California       LAC
2         Chicago               Illinois        CI
3         Houston                  Texas        HT
9    Jacksonville                Florida        JF
14      Charlotte         North Carolina       CNC
16  San Francisco             California       SFC
17        Seattle             Washington        SW
18         Denver               Colorado        DC
21     Washington   District of Columbia      WDOC

All Unique IDs:
New York,  New York: NYNY
Los Angeles,  California: LAC
Chicago,  Illinois: CI
Houston,  Texas: HT
Jacksonville,  Florida: JF
Charlotte,  North Carolina: CNC
San Francisco,  California: SFC
Seattle,  Washington: SW
Denver,  Colorado: DC
Washington,  District of Columbia: WDOC
Las Vegas,  Nevada: LVN
Boston,  Massachusetts: BM
Baltimore,  Maryland: BM1
Atlanta,  Georgia: AG
Kansas City,  Missouri: KCM
Miami,  Florida: M

In [12]:
#
nfl_city_info_df = pd.DataFrame(nfl_city_info_df)
new_order = ['Unique_ID', 'City', 'State', 'Population Est 2023']
nfl_city_info_df = nfl_city_info_df.reindex(columns=new_order)
nfl_city_info_df

,Unique_ID,City,State,Population Est 2023
0,NYNY,New York,New York,8258035
1,LAC,Los Angeles,California,3820914
2,CI,Chicago,Illinois,2664452
3,HT,Houston,Texas,2314157
9,JF,Jacksonville,Florida,985843
14,CNC,Charlotte,North Carolina,911311
16,SFC,San Francisco,California,808988
17,SW,Seattle,Washington,755078
18,DC,Denver,Colorado,716577
21,WDOC,Washington,District of Columbia,678972


In [13]:
# Save the updated DataFrame 
# nfl_city_info_df.to_csv('updated_nfl_cities.csv', index=False)